In [1]:
import torch
import torch.nn as nn
import typing
import csv

In [2]:
array = torch.zeros(3)


In [3]:
class Net(nn.Module):
    def __init__(self, numin, numout):
        super(Net, self).__init__()

        fc = []
        fc.append(nn.Linear(numin, numout))
        for i in range(20):
            fc.append(nn.Linear(numout, numout))
        self.fc = nn.ModuleList(fc)
        print(self.fc)

        self.relu = nn.ReLU()
        #self.sig = nn.Sigmoid()

    def forward(self, x):
        for fc in self.fc:
            x = fc(x)
            x = self.relu(x)
            # x = self.sig 
        return x

In [4]:
class Main:
    def start(self) -> int:
        device = 'cpu'
        # device = 'cuda'

        numInput = 400
        numOutput = 1420

        try:
            with open('data/input.csv', 'r', encoding='shift-jis') as f:
                reader = csv.reader(f)
                next(reader)  # skip the header
                inputs = []
                for row in reader:
                    v = []
                    for i in range(1, len(row)):
                        v.append(float(row[i]))
                    inputs.append(v)
        except FileNotFoundError as e:
            print(e)
            return 0
        '''
        for v in inputs:
            m = max(v)
            print(m)
        '''
        # print(max(inputs))
        # print(max(max(inputs)))
        # print(inputs)

        try:
            with open('data/target.csv', 'r',
                      encoding='shift-jis') as f:
                reader = csv.reader(f)
                next(reader)  # skip the header
                targets = []
                for row in reader:
                    v = []
                    for i in range(1, len(row)):
                        v.append(float(row[i]))
                    targets.append(v)
        except FileNotFoundError as e:
            print(e)
            return 0

        # print(len(inputs[0]))
        # print(len(targets[0]))

        trainInputs = torch.tensor(inputs)
        # print(trainInputs)
        trainInputs = trainInputs.to(device)

        trainTargets = torch.tensor(targets)
        # print(trainTargets)
        trainTargets = trainTargets.to(device)

        net: typing.Union[Net, torch.nn.DataParallel]
        net = Net(numInput, numOutput)
        if True and device != 'cpu':
            net = torch.nn.DataParallel(net)  # use multiple GPUs
        net = net.to(device)

        criterion = torch.nn.MSELoss()
        optimizer = torch.optim.Adam(net.parameters(), lr=0.01)

        for loop in range(1000):
            optimizer.zero_grad()

            trainOutputs = net(trainInputs)
            loss = criterion(trainOutputs, trainTargets)
            print(loss.item())

            loss.backward()
            optimizer.step()

        return 0

In [ ]:
Main().start()